In [1]:
using Luxor, LightXML;

In [2]:
drawing_height = 450
drawing_width = 800;

In [3]:
Drawing(drawing_width, drawing_height, :svg)
# box(O, Point(drawing_width, drawing_height), :stroke)
bbox = BoundingBox(;centered=false)

origin()

a_1 = 13/32*drawing_width
b_1 = 13/32*drawing_height
e_1 = sqrt(1 - b_1^2/a_1^2)
c_1 = a_1 * e_1
r_p_1 = a_1 * (1 - e_1)

m_1 = O + (c_1, 0)

a_2 = 5/16*drawing_width
b_2 = 5/16*drawing_height
e_2 = sqrt(1 - b_2^2/a_2^2)
c_2 = a_2 * e_2
center_2 = m_1 - (c_2, 0)

circle(m_1, 5, :fill)

ellipse(O, 2a_1, 2b_1, :stroke)
ellipse(center_2, 2a_2, 2b_2, :stroke)

origin(getworldposition(m_1, centered=false))

nu_A = 130  # degrees, outer orbit
nu_B = 160  # degrees, inner orbit
r_A = a_2 * (1 - e_2^2)/(1 + e_2 * cos(deg2rad(nu_A)))
r_B = a_1 * (1 - e_1^2)/(1 + e_1 * cos(deg2rad(nu_B)))
pt_A = polar(r_A, deg2rad(-nu_A))
pt_B = polar(r_B, deg2rad(-nu_B))
circle.([pt_A, pt_B], 5, :fill)

e_t = (r_A - r_B) / (r_B * cos(deg2rad(nu_B)) - r_A * cos(deg2rad(nu_A)))
p_t = r_A * r_B * (cos(deg2rad(nu_A)) - cos(deg2rad(nu_B))) / (r_A * cos(deg2rad(nu_A)) - r_B * cos(deg2rad(nu_B)))
a_t = p_t / (1 - e_t^2)
# c_t = a_t * e_t
# b_t = a_t * sqrt(1 - e_t^2)
# center_t = m_1 - (c_t, 0)
# f_t_1 = center_t + (c_t, 0)
# f_t_2 = center_t - (c_t, 0)

@layer begin
    setdash("dotted")
    rule(O, 0, boundingbox=bbox)
    line(O, pt_A, :stroke)
    line(O, pt_B, :stroke)
    setdash("solid")
    arrow(O, 4r_p_1 / 3, 0, deg2rad(-nu_A), clockwise=false)
    arrow(O, 13r_p_1 / 6, 0, deg2rad(-nu_B), clockwise=false)
end

transfer_angles = deg2rad.(nu_A:1:nu_B)
cairo_t_angles = deg2rad.(-nu_A:-1:-nu_B) # Cairo is clockwise
transfer_rs = a_t .* (1 - e_t^2) ./ (1 .+ e_t .* cos.(transfer_angles))

non_transfer_angles = deg2rad.(vcat(nu_B:1:360, 0:1:nu_A))
cairo_non_t_angles = deg2rad.(vcat(-nu_B:-1:-360, 0:-1:-nu_A))  # Cairo is clockwise
non_transfer_rs = a_t .* (1 - e_t^2) ./ (1 .+ e_t .* cos.(non_transfer_angles))

@layer begin
    setcolor(141/255, 160/255, 203/255, 1.0)
    poly(polar.(transfer_rs, cairo_t_angles), :stroke)

    setdash("dashed")
    poly(polar.(non_transfer_rs, cairo_non_t_angles), :stroke)
    
    triangle_index = 2*(length(transfer_angles) ÷ 3)
    triangle_point = polar(transfer_rs[triangle_index], cairo_t_angles[triangle_index])
    ngon(triangle_point, 7, 3, pi, :fill)
end

fontface("Dejavu Sans")
fontsize(20)
# textwrap("Apse Line", 100, O)
label("A", :NE, pt_A)
label("B", :N, pt_B)
finish()
svgout = svgstring()
xdoc = parse_string(svgout)
xroot = root(xdoc)
save_file(xdoc, "../raw_svg/common-apse-line-transfer.svg");
